<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/tensorflow-1-public/blob/main/C2/W1/ungraded_lab/C2_W1_Lab_1_cats_vs_dogs_Traducida.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Laboratorio no calificado: Uso de imágenes más sofisticadas con Redes Neuronales Convolucionales

En el curso 1 de esta especialización, viste cómo usar una CNN para hacer más eficiente el reconocimiento de imágenes generadas por computadora de caballos y humanos. En esta lección, pasarás al siguiente nivel: construirás un modelo para clasificar imágenes reales de gatos y perros. Al igual que el conjunto de datos de caballos y humanos, las imágenes del mundo real también tienen diferentes formas, relaciones de aspecto, etc. y tendrás que tenerlo en cuenta al preparar tus datos.

En este laboratorio, primero revisarás cómo construir CNNs, prepararás tus datos con `ImageDataGenerator` y examinarás tus resultados. Seguirás estos pasos:

1.   Explorar los datos de ejemplo de `Perros vs. Gatos`.
2.   Construir y entrenar una red neuronal para clasificar entre las dos mascotas
3.   Evaluar la precisión del entrenamiento y la validación

En los próximos laboratorios se basará en los resultados obtenidos aquí para mejorarlos, especialmente para evitar el sobreajuste. ¡Comencemos!
    
**NOTA IMPORTANTE:** Este cuaderno está diseñado para ejecutarse como Colab. Ejecutarlo en su máquina local puede dar lugar a que algunos de los bloques de código arrojen errores.

<details><summary>Texto Original</summary>

# Ungraded Lab: Using more sophisticated images with Convolutional Neural Networks

In Course 1 of this specialization, you saw how to use a CNN to make your recognition of computer generated images of horses and humans more efficient. In this lesson, you'll take that to the next level: building a model to classify real images of cats and dogs. Like the horses and humans dataset, real-world images also come in different shapes, aspect ratios, etc. and you will need to take this into account when preparing your data.

In this lab, you will first review how to build CNNs, prepare your data with `ImageDataGenerator` and examine your results. You'll follow these steps:

1.   Explore the example data of `Dogs vs. Cats`
2.   Build and train a neural network to classify between the two pets
3.   Evaluate the training and validation accuracy

You will build upon your results here in the next labs so you can improve it, particularly in avoiding overfitting. Let's begin!
    
**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Running it on your local machine might result in some of the code blocks throwing errors.

In [ ]:
# Install this package to use Colab's GPU for training
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

## Descarga e inspección del conjunto de datos

Comenzará descargando el conjunto de datos. Se trata de un `.zip` de 2.000 imágenes JPG de perros y gatos. Es un subconjunto del conjunto de datos ["Dogs vs. Cats"](https://www.kaggle.com/c/dogs-vs-cats/data) disponible en Kaggle, que contiene 25.000 imágenes. Sólo se utilizarán 2.000 del conjunto de datos completo para reducir el tiempo de entrenamiento con fines educativos.


<details><summary>Texto Original</summary>

## Download and Inspect the Dataset

You will start by downloading the dataset. This is a `.zip` of 2,000 JPG pictures of cats and dogs. It is a subset of the ["Dogs vs. Cats" dataset](https://www.kaggle.com/c/dogs-vs-cats/data) available on Kaggle, which contains 25,000 images. You will only use 2,000 of the full dataset to decrease training time for educational purposes.

In [ ]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

A continuación, lo extraerá al directorio actual.


<details><summary>Texto Original</summary>

You will then extract it to the current directory.

In [ ]:
import zipfile

# Unzip the archive
local_zip = './cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()

El contenido del .zip se extrae al directorio base `./gatos_y_perros_filtrados`, que contiene los subdirectorios `train` y `validation` para los conjuntos de datos de entrenamiento y validación (puedes ignorar `vectorize.py` en la salida de la siguiente celda). 

Si recuerdas, el **conjunto de entrenamiento** son los datos que se utilizan para decirle al modelo de la red neuronal que "este es el aspecto de un gato" y "este es el aspecto de un perro". El **conjunto de validación** son imágenes de gatos y perros que la red neuronal no verá como parte del entrenamiento. Puedes utilizarlo para probar lo bien o lo mal que lo hace al evaluar si una imagen contiene un gato o un perro. (Consulta el [Curso acelerado de aprendizaje automático](https://developers.google.com/machine-learning/crash-course/validation/check-your-intuition) si quieres un repaso de los conjuntos de entrenamiento, validación y prueba).

Estos subdirectorios contienen a su vez subdirectorios `gatos` y `perros`.


<details><summary>Texto Original</summary>


The contents of the .zip are extracted to the base directory `./cats_and_dogs_filtered`, which contains `train` and `validation` subdirectories for the training and validation datasets (you can ignore `vectorize.py` in the output in the next cell). 

If you recall, the **training set** is the data that is used to tell the neural network model that 'this is what a cat looks like' and 'this is what a dog looks like'. The **validation set** is images of cats and dogs that the neural network will not see as part of the training. You can use this to test how well or how badly it does in evaluating if an image contains a cat or a dog. (See the [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/validation/check-your-intuition) if you want a refresher on training, validation, and test sets.)

These subdirectories in turn each contain `cats` and `dogs` subdirectories.

In [ ]:
import os

base_dir = 'cats_and_dogs_filtered'

print("Contents of base directory:")
print(os.listdir(base_dir))

print("\nContents of train directory:")
print(os.listdir(f'{base_dir}/train'))

print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}/validation'))

Puedes asignar cada uno de estos directorios a una variable para poder utilizarlo posteriormente.


<details><summary>Texto Original</summary>

You can assign each of these directories to a variable so you can use it later.

In [ ]:
import os

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


Ahora vea cómo son los nombres de los archivos en los directorios `cats` y `dogs` `train` (las convenciones de nomenclatura de los archivos son las mismas en el directorio `validation`):

<details><summary>Texto Original</summary>

Now see what the filenames look like in the `cats` and `dogs` `train` directories (file naming conventions are the same in the `validation` directory):

In [ ]:
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

Averigüemos el número total de imágenes de gatos y perros en los directorios `train` y `validation`:

<details><summary>Texto Original</summary>

Let's find out the total number of cat and dog images in the `train` and `validation` directories:

In [ ]:
print('total training cat images :', len(os.listdir(      train_cats_dir ) ))
print('total training dog images :', len(os.listdir(      train_dogs_dir ) ))

print('total validation cat images :', len(os.listdir( validation_cats_dir ) ))
print('total validation dog images :', len(os.listdir( validation_dogs_dir ) ))

Tanto para los gatos como para los perros, tienes 1.000 imágenes de entrenamiento y 500 imágenes de validación.

Ahora echa un vistazo a algunas imágenes para tener una mejor idea de cómo son los conjuntos de datos de gatos y perros. Primero, configure los parámetros de `matplotlib`:

<details><summary>Texto Original</summary>

For both cats and dogs, you have 1,000 training images and 500 validation images.

Now take a look at a few pictures to get a better sense of what the cat and dog datasets look like. First, configure the `matplotlib` parameters:

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images

Ahora, muestre un lote de 8 fotos de gatos y 8 de perros. Puede volver a ejecutar la celda para ver un nuevo lote cada vez:

<details><summary>Texto Original</summary>

Now, display a batch of 8 cat and 8 dog pictures. You can re-run the cell to see a fresh batch each time:

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_cat_pix = [os.path.join(train_cats_dir, fname) 
                for fname in train_cat_fnames[ pic_index-8:pic_index] 
               ]

next_dog_pix = [os.path.join(train_dogs_dir, fname) 
                for fname in train_dog_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


Puede que no resulte obvio al ver las imágenes de esta cuadrícula, pero una nota importante es que estas imágenes tienen todas las formas y tamaños (al igual que el conjunto de datos "caballos o humanos"). Así que antes de entrenar una red neuronal con ellas, tendrás que ajustar las imágenes. Lo verás en las siguientes secciones.
    
## Construir un pequeño modelo desde cero para llegar a una precisión del 72%.

Para entrenar una red neuronal para manejar las imágenes, necesitarás que tengan un tamaño uniforme. Elegirás 150x150 píxeles para esto, y verás el código que preprocesa las imágenes a esa forma en breve. 

Puedes definir el modelo importando Tensorflow y utilizando la API de Keras. Aquí está el código completo primero y la discusión viene después. Esto es muy similar a los modelos que has construido en el Curso 1.

<details><summary>Texto Original</summary>

It may not be obvious from looking at the images in this grid but an important note here is that these images come in all shapes and sizes (just like the 'horses or humans' dataset). So before training a neural network with them, you'll need to tweak the images. You'll see that in the next sections.
    
## Building a Small Model from Scratch to get to ~72% Accuracy

To train a neural network to handle the images, you'll need them to be in a uniform size. You will choose 150x150 pixels for this, and you'll see the code that preprocesses the images to that shape shortly. 

You can define the model by importing Tensorflow and using the Keras API. Here is the entire code first then the discussion comes after. This is very similar to the models you have built in Course 1.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

Ha definido una capa `Secuencial` como antes, añadiendo primero algunas capas convolucionales. Esta vez, observe el parámetro "input_shape". Aquí es donde pones el tamaño `150x150` y `3` para la profundidad de color porque tienes imágenes coloreadas. A continuación, agrega un par de capas convolucionales y aplana el resultado final para alimentar las capas densamente conectadas.
    
Ten en cuenta que como te enfrentas a un problema de clasificación de dos clases, es decir, un *problema de clasificación binaria*, terminarás la red con una [activación *sigmoidea*](https://wikipedia.org/wiki/Sigmoid_function). La salida de la red será un único escalar entre `0` y `1`, que codifica la probabilidad de que la imagen actual sea de clase `1` (en contraposición a la clase `0`).
    
Puede revisar la arquitectura de la red con el método `model.summary()`: 

<details><summary>Texto Original</summary>

You defined a `Sequential` layer as before, adding some convolutional layers first. Note the `input_shape` parameter this time. Here is where you put the `150x150` size and `3` for the color depth because you have colored images. You then add a couple of convolutional layers and flatten the final result to feed into the densely connected layers.
    
Note that because you are facing a two-class classification problem, i.e. a *binary classification problem*, you will end the network with a [*sigmoid* activation](https://wikipedia.org/wiki/Sigmoid_function). The output of the network will be a single scalar between `0` and `1`, encoding the probability that the current image is class `1` (as opposed to class `0`).
    
You can review the architecture of the network with the `model.summary()` method: 

In [ ]:
model.summary()

Puede revisar la arquitectura de la red con el método `model.summary()`: 

La columna `output_shape` muestra cómo evoluciona el tamaño de su mapa de características en cada capa sucesiva. La operación de convolución elimina los píxeles más externos de las dimensiones originales, y cada capa de agrupación la reduce a la mitad.

A continuación, configurará las especificaciones para el entrenamiento del modelo. Entrenaremos nuestro modelo con la pérdida `binary_crossentropy`, porque es un problema de clasificación binaria y su activación final es una sigmoide. Usaremos el optimizador `rmsprop` con una tasa de aprendizaje de `0.001`. Durante el entrenamiento, usted querrá monitorear la precisión de la clasificación.

**NOTA**: En este caso, el uso del [algoritmo de optimización RMSprop](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) es preferible al [descenso de gradiente estocástico](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), porque RMSprop automatiza el ajuste de la tasa de aprendizaje para nosotros. (Otros optimizadores, como [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) y [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), también adaptan automáticamente la tasa de aprendizaje durante el entrenamiento, y funcionarían igualmente bien aquí).

<details><summary>Texto Original</summary>

You can review the architecture of the network with the `model.summary()` method: 

The `output_shape` column shows how the size of your feature map evolves in each successive layer. The convolution operation removes the outermost pixels from the original dimensions, and each pooling layer halves it.

Next, you'll configure the specifications for model training. You will train our model with the `binary_crossentropy` loss, because it's a binary classification problem and your final activation is a sigmoid. We will use the `rmsprop` optimizer with a learning rate of `0.001`. During training, you will want to monitor classification accuracy.

**NOTE**: In this case, using the [RMSprop optimization algorithm](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) is preferable to [stochastic gradient descent](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), because RMSprop automates learning-rate tuning for us. (Other optimizers, such as [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) and [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), also automatically adapt the learning rate during training, and would work equally well here.)

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

### Preprocesamiento de datos

El siguiente paso es configurar los generadores de datos que leerán las imágenes en las carpetas de origen, las convertirán en tensores `float32` y las alimentarán (con sus etiquetas) al modelo. Tendrás un generador para las imágenes de entrenamiento y otro para las de validación. Estos generadores producirán lotes de imágenes de tamaño 150x150 y sus etiquetas (binarias).

Como ya sabrás, los datos que se introducen en las redes neuronales deben ser normalizados de alguna manera para que sean más fáciles de procesar por la red (por ejemplo, no es común introducir píxeles en bruto en una ConvNet). En este caso, preprocesarás las imágenes normalizando los valores de los píxeles para que estén en el rango `[0, 1]` (originalmente todos los valores están en el rango `[0, 255]`).

En Keras, esto se puede hacer a través de la clase `keras.preprocessing.image.ImageDataGenerator` utilizando el parámetro `rescale`. Esta clase `ImageDataGenerator` permite instanciar generadores de lotes de imágenes aumentadas (y sus etiquetas) mediante `.flow(data, labels)` o `.flow_from_directory(directory)`.

<details><summary>Texto Original</summary>

### Data Preprocessing

Next step is to set up the data generators that will read pictures in the source folders, convert them to `float32` tensors, and feed them (with their labels) to the model. You'll have one generator for the training images and one for the validation images. These generators will yield batches of images of size 150x150 and their labels (binary).

As you may already know, data that goes into neural networks should usually be normalized in some way to make it more amenable to processing by the network (i.e. It is uncommon to feed raw pixels into a ConvNet.) In this case, you will preprocess the images by normalizing the pixel values to be in the `[0, 1]` range (originally all values are in the `[0, 255]` range).

In Keras, this can be done via the `keras.preprocessing.image.ImageDataGenerator` class using the `rescale` parameter. This `ImageDataGenerator` class allows you to instantiate generators of augmented image batches (and their labels) via `.flow(data, labels)` or `.flow_from_directory(directory)`.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))


### Entrenamiento
Ahora entrenará en las 2.000 imágenes disponibles, durante 15 épocas, y controlará la precisión también en las 1.000 imágenes del conjunto de validación.

Observe los valores por época.

Verá 4 valores por época - Pérdida, Precisión, Pérdida de Validación y Precisión de Validación. 

La `pérdida` y la `precisión` son grandes indicadores del progreso en el entrenamiento. La `pérdida` mide la predicción del modelo actual frente a las etiquetas conocidas, calculando el resultado. Por otro lado, la "precisión" es la proporción de aciertos. 


<details><summary>Texto Original</summary>

### Training
You will now train on all 2,000 images available, for 15 epochs, and monitor the accuracy as well on the 1,000 images in the validation set.

Do note the values per epoch.

You'll see 4 values per epoch -- Loss, Accuracy, Validation Loss and Validation Accuracy. 

The `loss` and `accuracy` are great indicators of progress in training. `loss` measures the current model prediction against the known labels, calculating the result. `accuracy`, on the other hand, is the portion of correct guesses. 


In [ ]:
history = model.fit(
            train_generator,
            epochs=15,
            validation_data=validation_generator,
            verbose=2
            )

### Predicción del modelo

Ahora eche un vistazo a la ejecución de una predicción utilizando el modelo. Este código le permitirá elegir 1 o más archivos de su sistema de archivos, cargarlos y ejecutarlos a través del modelo, dando una indicación de si el objeto es un gato o un perro.

**Nota importante:** Debido a algunos problemas de compatibilidad, el siguiente bloque de código dará lugar a un error después de seleccionar las imágenes a cargar si está ejecutando este cuaderno como un `Colab` en el navegador `Safari`. Para todos los demás navegadores, continúe con el siguiente bloque de código e ignore el que le sigue.

Para los usuarios de Safari: por favor, comente u omita el bloque de código de abajo, descomente el siguiente bloque de código y ejecútelo.

<details><summary>Texto Original</summary>

### Model Prediction

Now take a look at actually running a prediction using the model. This code will allow you to choose 1 or more files from your file system, upload them, and run them through the model, giving an indication of whether the object is a cat or a dog.

**Important Note:** Due to some compatibility issues, the following code block will result in an error after you select the images(s) to upload if you are running this notebook as a `Colab` on the `Safari` browser. For all other browsers, continue with the next code block and ignore the next one after it.

_For Safari users: please comment out or skip the code block below, uncomment the next code block and run it._

In [ ]:
## CODE BLOCK FOR NON-SAFARI BROWSERS
## SAFARI USERS: PLEASE SKIP THIS BLOCK AND RUN THE NEXT ONE INSTEAD

import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
    # predicting images
    path='/content/' + fn
    img=tf.keras.utils.load_img(path, target_size=(150, 150))

    x=tf.keras.utils.img_to_array(img)
    x /= 255
    x=np.expand_dims(x, axis=0)
    images = np.vstack([x])

    classes = model.predict(images, batch_size=10)

    print(classes[0])

    if classes[0]>0.5:
        print(fn + " is a dog")
    else:
        print(fn + " is a cat")
 

Los usuarios de `Safari` tendrán que cargar las imágenes manualmente en su espacio de trabajo. Por favor, siga las instrucciones, descomente el bloque de código de abajo y ejecútelo.

Instrucciones para subir imágenes manualmente en un Colab:

1. Seleccione el icono de la `carpeta` en la `barra de menú` de la izquierda.
2. Haga clic en la `carpeta con una flecha apuntando hacia arriba` llamada `..`.
3. Haga clic en la `carpeta` llamada `tmp`.
4. Dentro de la carpeta `tmp`, `crea una nueva carpeta` llamada `imágenes`. Verás la opción `Nueva carpeta` haciendo clic en el botón de menú de los `3 puntos verticales` junto a la carpeta `tmp`.
5. 5. Dentro de la nueva carpeta `imágenes`, sube una o varias imágenes de tu elección, preferiblemente de un gato o un perro. Arrastra y suelta la(s) imagen(es) encima de la carpeta `images`.
6. Descomente y ejecute el bloque de código que aparece a continuación. 

<details><summary>Texto Original</summary>

`Safari` users will need to upload the images(s) manually in their workspace. Please follow the instructions, uncomment the code block below and run it.

Instructions on how to upload image(s) manually in a Colab:

1. Select the `folder` icon on the left `menu bar`.
2. Click on the `folder with an arrow pointing upwards` named `..`
3. Click on the `folder` named `tmp`.
4. Inside of the `tmp` folder, `create a new folder` called `images`. You'll see the `New folder` option by clicking the `3 vertical dots` menu button next to the `tmp` folder.
5. Inside of the new `images` folder, upload an image(s) of your choice, preferably of either a cat or a dog. Drag and drop the images(s) on top of the `images` folder.
6. Uncomment and run the code block below. 

In [ ]:
# # CODE BLOCK FOR SAFARI USERS

# import numpy as np
# from keras.preprocessing import image
# import os

# images = os.listdir("/tmp/images")

# print(images)

# for i in images:
#  print()
#  # predicting images
#  path = '/tmp/images/' + i
#  img = image.load_img(path, target_size=(150, 150))
#  x = image.img_to_array(img)
#  x /= 255
#  x = np.expand_dims(x, axis=0)

#  images = np.vstack([x])
#  classes = model.predict(images, batch_size=10)
#  print(classes[0])
#  if classes[0]>0.5:
#    print(i + " is a dog")
#  else:
#    print(i + " is a cat")

### Visualización de representaciones intermedias

Para tener una idea de qué tipo de características ha aprendido tu CNN, una cosa divertida es visualizar cómo se transforma una entrada a medida que pasa por el modelo.

Puedes elegir una imagen al azar del conjunto de entrenamiento, y luego generar una figura donde cada fila es la salida de una capa, y cada imagen en la fila es un filtro específico en ese mapa de características de salida. Vuelva a ejecutar esta celda para generar representaciones intermedias para una variedad de imágenes de entrenamiento.

<details><summary>Texto Original</summary>

### Visualizing Intermediate Representations

To get a feel for what kind of features your CNN has learned, one fun thing to do is to visualize how an input gets transformed as it goes through the model.

You can pick a random image from the training set, and then generate a figure where each row is the output of a layer, and each image in the row is a specific filter in that output feature map. Rerun this cell to generate intermediate representations for a variety of training images.

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model
successive_outputs = [layer.output for layer in model.layers]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Prepare a random input image from the training set.
cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]
img_path = random.choice(cat_img_files + dog_img_files)
img = load_img(img_path, target_size=(150, 150))  # this is a PIL image
x   = img_to_array(img)                           # Numpy array with shape (150, 150, 3)
x   = x.reshape((1,) + x.shape)                   # Numpy array with shape (1, 150, 150, 3)

# Scale by 1/255
x /= 255.0

# Run the image through the network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so you can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Display the representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    #-------------------------------------------
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    #-------------------------------------------
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # Tile the images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
        x  = feature_map[0, :, :, i]
        x -= x.mean()
        x /= x.std ()
        x *=  64
        x += 128
        x  = np.clip(x, 0, 255).astype('uint8')
        display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display the grid
    #-----------------
    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' ) 

Se puede ver arriba cómo los píxeles resaltados se convierten en representaciones cada vez más abstractas y compactas, especialmente en la red inferior. 

Las representaciones de la parte inferior empiezan a resaltar aquello a lo que la red presta atención, y muestran cada vez menos características que se "activan"; la mayoría se ponen a cero. Esto se llama _representation sparsity_ y es una característica clave del aprendizaje profundo. Estas representaciones llevan cada vez menos información sobre los píxeles originales de la imagen, pero una información cada vez más refinada sobre la clase de la imagen. Se puede pensar en una convnet (o en una red profunda en general) como una tubería de destilación de información en la que cada capa filtra las características más útiles.
    
### Evaluación de la precisión y la pérdida del modelo

Se trazará la precisión y la pérdida del entrenamiento/validación como se recoge durante el entrenamiento:


<details><summary>Texto Original</summary>

You can see above how the pixels highlighted turn to increasingly abstract and compact representations, especially at the bottom grid. 

The representations downstream start highlighting what the network pays attention to, and they show fewer and fewer features being "activated"; most are set to zero. This is called _representation sparsity_ and is a key feature of deep learning. These representations carry increasingly less information about the original pixels of the image, but increasingly refined information about the class of the image. You can think of a convnet (or a deep network in general) as an information distillation pipeline wherein each layer filters out the most useful features.
    
### Evaluating Accuracy and Loss for the Model

You will plot the training/validation accuracy and loss as collected during training:

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

Como se puede ver, el modelo se está **sobreajustando** como si estuviera pasando de moda. La precisión de entrenamiento (en azul) se acerca al 100% mientras que la precisión de validación (en naranja) se estanca en el 70%. La pérdida de validación alcanza su mínimo después de sólo cinco épocas.

Dado que tiene un número relativamente pequeño de ejemplos de entrenamiento (2000), la sobreadaptación debería ser la principal preocupación. La sobreadaptación se produce cuando un modelo expuesto a muy pocos ejemplos aprende patrones que no se generalizan a los nuevos datos, es decir, cuando el modelo empieza a utilizar características irrelevantes para hacer predicciones. Por ejemplo, si usted, como humano, sólo ve tres imágenes de personas que son leñadores y tres imágenes de personas que son marineros, y entre ellas la única persona que lleva una gorra es un leñador, podría empezar a pensar que llevar una gorra es un signo de ser un leñador en contraposición a un marinero. En ese caso, sería un pésimo clasificador de leñadores y marineros.

La sobreadaptación es el problema central del aprendizaje automático: dado que estamos ajustando los parámetros de nuestro modelo a un conjunto de datos determinado, ¿cómo podemos asegurarnos de que las representaciones aprendidas por el modelo serán aplicables a datos que nunca ha visto antes? ¿Cómo evitar que se aprendan cosas específicas de los datos de entrenamiento?

En el siguiente ejercicio, verás formas de evitar el sobreajuste en este modelo de clasificación.
    
## Limpieza

Antes de ejecutar el siguiente ejercicio, ejecuta la siguiente celda para terminar el kernel y liberar recursos de memoria:

<details><summary>Texto Original</summary>

As you can see, the model is **overfitting** like it's getting out of fashion. The training accuracy (in blue) gets close to 100% while the validation accuracy (in orange) stalls as 70%. The validation loss reaches its minimum after only five epochs.

Since you have a relatively small number of training examples (2000), overfitting should be the number one concern. Overfitting happens when a model exposed to too few examples learns patterns that do not generalize to new data, i.e. when the model starts using irrelevant features for making predictions. For instance, if you, as a human, only see three images of people who are lumberjacks, and three images of people who are sailors, and among them the only person wearing a cap is a lumberjack, you might start thinking that wearing a cap is a sign of being a lumberjack as opposed to a sailor. You would then make a pretty lousy lumberjack/sailor classifier.

Overfitting is the central problem in machine learning: given that you are fitting the parameters of our model to a given dataset, how can you make sure that the representations learned by the model will be applicable to data it has never seen before? How do you avoid learning things that are specific to the training data?

In the next exercise, you'll look at ways to prevent overfitting in this classification model.
    
## Clean Up

Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [ ]:
import os, signal

os.kill(     os.getpid() , 
         signal.SIGKILL
       )